### Step 1: Define directories and specify plates to image
Ensure the following before running anything in this notebook:
 - Plates with duckweed are loaded in the correct positions on the bedplate
 - The machine is switched on, connected to the computer and all axes have been homed. 
 - The camera tool is accessible and in position 1 on the Jubilee

In [25]:
#Define directory to save image data for this experiment
data_dir = '/home/pi/Documents/Data'
expt_dir = data_dir + '/Test_expt'
expt_id = "test"
print(expt_dir)

/home/pi/Documents/Data/Test_expt


In [ ]:
#Which plates do you want to image (numbers correspond to positions on bedplate. Options: 1,2,3,4,5)
plates_to_image = [1]

### Step 2: Run these cells to import required packages and define methods

In [31]:
import serial 
from serial.tools import list_ports
import yaml
import matplotlib
matplotlib.use('TkAgg') #This is a specific back end of matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2
import time
import glob
import json
import datetime
from datetime import date
from MachineUtils import *
from CameraUtils import *
import Plate_positions as pp


In [32]:
def move_across_platewells(well_coord_list_of_dicts):
    for i, well in enumerate(well_coord_list_of_dicts):
        m.moveTo(x=well["x"], y=well["y"], z=10)
        ser.write(b'G4 P1000') 

def image_plate(plate_num):
    well_coord_list_of_dicts = pp.fetch_plate_wellpostions(plate_num)
    for i, well in enumerate(well_coord_list_of_dicts):
        m.moveTo(x=well["x"], y=well["y"], z=10)
#         wellFrame, wellData = getSingleWell(getFrame(), minR=330, maxR=335) #Adjust focus
#         well = wellData[0]
#         print(well)
        f = getFrame()
        cv2.imwrite(f'{expt_dir}/{expt_id}_plate{plate_num}_well{well["well_id"]}_{date.today()}.jpg', f)

In [4]:
ports = serial.tools.list_ports.comports()
ser = serial.Serial('/dev/ttyACM0', 115200) #Port address and Baudrate (info transfer rate)
port = '/dev/ttyACM0'
m = MachineCommunication(port)
ser.write(b'G90\n') #Command to make sure the Jubilee is using absolute as opposed to relative positioning
m.toolChange(1)

['ttyACM0', 'ttyAMA0']


### Step 3: Run this cell to image the plates defined in step 1 

In [30]:
for p in plates_to_image:
    image_plate(p)

... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
... camera connection established
